In [1]:
import notebook_importer

In [2]:
import spdz
import random
import numpy as np
import zmq
import time

importing notebook from spdz.ipynb


In [3]:
# Alice is party 0
party = 0

# Connect to zmq
context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect ("tcp://localhost:5555")

In [4]:
# seed random numbers to make calculation
# deterministic (just a good practice)
random.seed(1)
np.random.seed(1)

# initialize weights randomly with mean 0
syn0 = 2*np.random.random((3,1)) - 1
syn0_shares = spdz.secure(syn0)

# Split weights into two shares
syn0_alice = spdz.get_share(syn0_shares, 0)
syn0_bob = spdz.get_share(syn0_shares, 1)

In [5]:
# Alice sends initial weights to Bob
print(spdz.swap_shares(syn0_bob, party, socket))

# Alice receives X and y data from Bob
X_alice = spdz.swap_shares(np.array("OK"), party, socket)
y_alice = spdz.swap_shares(np.array("OK"), party, socket)

OK


In [6]:
# Train in sync with Bob

#sigmoid = spdz.SigmoidInterpolated10()
sigmoid = spdz.Sigmoid()

network = spdz.TwoLayerNetwork(sigmoid, party, socket)

start_time = time.time()
network.train(X_alice, y_alice, syn0_alice)

print("Elapsed Training Time:\n", time.time() - start_time)
network.print_weights()

Elapsed Training Time:
 49.09345006942749
Layer 0 weights: 
[[ 9.6731805]
 [-0.2077667]
 [-4.6290403]]
